<a href="https://colab.research.google.com/github/greg881203/-/blob/main/people.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import tensorflow as tf
import h5py
from skimage.io import imread
import os
import cv2

# file = os.listdir('data/images')
# # print(file)
# print(len(file))

imgs = []
ground_truths = []
file = os.listdir('data/images')
grt = os.listdir('data/grounthtruth')

for i in range(len(file)):
  img_path = f'data/images/{file[i]}'
  img = imread(img_path)
#   img = np.expand_dims(img, axis=0)
  imgs.append(img)

  gt_path = f'data/grounthtruth/{grt[i]}'
  h5_file = h5py.File(gt_path, 'r')
  ground_truth = np.array(h5_file['density'])
  resized_ground_truth = cv2.resize(ground_truth, (120, 68),interpolation=cv2.INTER_CUBIC)* (16**2)
  ground_truths.append(resized_ground_truth)


# imgs = np.concatenate(imgs, axis=0)
# print(imgs.shape)

imgs = np.array(imgs, dtype=np.float32) / 255.0
dataset = tf.data.Dataset.from_tensor_slices((imgs, ground_truths))
print(dataset)
train_dataset = dataset.batch(1)
print(train_dataset)

ValueError: ignored

In [1]:
# import shutil

# # 設定要刪除檔案的資料夾路徑
# folder_path = 'data/grounthtruth'

# # 使用 shutil.rmtree 刪除整個資料夾及其內容
# shutil.rmtree(folder_path)

In [ ]:
file = os.listdir('data/gt')
# print(file)
print(len(file))

299


In [ ]:
# 總體結構
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
# 圖像輸入
img_input = Input(shape=(1080, 1920, 3))

# 使用MobileNetV2作為特徵提取層
mobilenetv2 = MobileNetV2(include_top=False, weights='imagenet', input_tensor=img_input)

x = mobilenetv2.layers[115].output
# x = mobilenetv2.layers[115].output
# 10753
x = Conv2D(filters=96, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=1,groups=96,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=1, groups=96,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=1, groups=192,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=1, groups=192,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=1, groups=192,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=1, groups=192,activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, strides=(1, 1), padding="same", dilation_rate=1, groups=192,activation='relu')(x)

# 最後輸出density map
density_map = Conv2D(filters=1, kernel_size=1, name='density_map')(x)

# 建立模型
model = Model(img_input, density_map)
model.summary()

In [ ]:

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_dataset, batch_size=1,verbose=1,epochs=100)

In [ ]:
img = imread('1.jpg')
imgs = np.array(img, dtype=np.float32) / 255.0

density_map = model.predict(np.expand_dims(imgs, axis=0))
# print("預測:", density_map)
print("預測 shape :", density_map.shape)
# # 創建密度圖

density_mapsq = density_map.squeeze()  # 這裡替換為你的密度圖

print("預測 density_map squeeze shape :", density_mapsq.shape)
print("predict Count : ", np.around(np.sum(density_mapsq)))
# 繪製等高線圖
plt.imshow(density_mapsq, cmap=cm.jet)
plt.colorbar()
plt.show()